In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra
using Pkg
using IJulia
ENV["COLUMNS"] = 200

# Assuming your Julia file is named "COVIDResourceAllocation.jl"
include("COVIDResourceAllocation.jl")
using .COVIDResourceAllocation

states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0

pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours)


forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
)

demand = forecast_active * Float32(nurse_days_per_patient_day_covid)

nurses = n_nurses(states) * Float32(pct_nurses_available)

_nurses_beds = n_nurses(states, source=:beds);
_nurses_empl = n_nurses(states, source=:employment);
_nurses_ahrf = n_nurses(states, source=:ahrf);
DataFrame(
    state = states,
    from_beds = _nurses_beds,
    from_empl = _nurses_empl,
    from_ahrf = _nurses_ahrf,
)

# model = reusable_resource_allocation(
#     nurses,
#     zeros(Float32, size(demand)...),
#     demand,
#     adj,
#     send_new_only=false,
#     sendrecieve_switch_time=0,
#     min_send_amt=0,
#     smoothness_penalty=0,
#     setup_cost=0,
#     sent_penalty=0.01,
#     verbose=true
# )
# sent = value.(model[:sent])
# println("termination status: ", termination_status(model))
# println("solve time: ", round(solve_time(model), digits=3), "s")
# println("objective function value: ", round(objective_value(model), digits=3))

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


Row,state,from_beds,from_empl,from_ahrf
,String,Float32,Float32,Float32
1,CT,8221.27,12120.0,4593.0
2,DE,2257.27,3234.06,1322.0
3,MA,17563.5,38220.5,10146.0
4,MD,9598.4,29581.0,6169.0
5,ME,2877.47,2346.56,1962.0
6,NH,2567.13,1484.33,1934.0
7,NJ,19164.1,71498.5,6652.0
8,NY,44213.9,83342.8,19547.0
9,PA,34385.9,65427.3,14189.0


In [2]:
supply

UndefVarError: UndefVarError: `supply` not defined